# ANN to model Cl

In [4]:
# Create dataset
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import torch

results_dir = "results/"


class AerofoilLiftSample:
    def __init__(self, file_name):
        self.file_name = file_name
        aerofoil_data = self.file_name.split("_")
        naca = aerofoil_data[1]
        self.camber = naca[0]
        self.camber_pos = naca[1]
        self.thickness = naca[2:3]
        self.reynolds = aerofoil_data[2]
        self.mach = aerofoil_data[3]
        self.alpha = aerofoil_data[4].split(".")[0]

    def __str__(self):
        return self.file_name

    def tensor(self):
        return torch.tensor([float(self.camber),
                             float(self.camber_pos),
                             float(self.thickness),
                             float(self.reynolds),
                             float(self.mach),
                             float(self.alpha),
                             float(self.cl())],
                            dtype=torch.int32)

    def cp_data(self):
        file = results_dir + self.file_name
        df = pd.read_csv(file, sep="\s+", skiprows=[0,1,2], names=["x", "y", "Cp"], dtype=float)
        return df

    def cl(self):
        return np.trapz(self.cp_data()["Cp"], self.cp_data()["x"])

class AerofoilLiftDataset(Dataset):
    """Aerofoil Coefficient of Lift Dataset"""
    def __init__(self):
        self.samples = self.load_samples()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, item):
        return self.samples[item].tensor()

    def load_samples(self):
        samples = []
        for file in os.listdir(results_dir):
            samples.append(AerofoilLiftSample(file))
        return samples

In [7]:
from torch.utils.data import DataLoader, random_split

dataset = AerofoilLiftDataset()
print(dataset[0])

train_size = int(0.6 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataLoader = DataLoader(train_dataset, batch_size=128,shuffle=True)
test_dataLoader = DataLoader(test_dataset, batch_size=128, shuffle=True)

tensor([      0,       0,       0, 1000000,       0,      -2,       0],
       dtype=torch.int32)


C:\Users\Sam\AppData\Local\Temp/ipykernel_10284/2198002848.py:27: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.tensor([float(self.camber),
